# Chapter 6. 요약

In [1]:
import sys
print(sys.version)

3.10.11 (main, Apr 20 2023, 19:02:41) [GCC 11.2.0]


In [2]:
!nvidia-smi

Thu Jun 29 23:45:32 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 515.105.01   Driver Version: 515.105.01   CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:01:00.0  On |                  N/A |
|  0%   38C    P8    22W / 170W |   2196MiB / 12288MiB |     17%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [6]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# GPU 메모리 사용량 확인
if device.type =="cuda":
    allocated_memory = torch.cuda.memory_allocated(device = device)
    cached_memory = torch.cuda.memory_reserved(device = device)

print(f"Allocated Memory:{allocated_memory / 1024 ** 2:.2f} MB")
print(f"Cached Memory: {cached_memory / 1024**2:.2f} MB")

Allocated Memory:0.00 MB
Cached Memory: 0.00 MB


**(서론)**

언젠가 한 번쯤 문서를 요약해야 할 때가 있습니다. 요약할 문서는 연구 논문이나 재무 실적 보고서 아니면 이메일 스레드일지도 모릅니다. 생각해보면 이런 작업에는 긴 단락을 이해하고, 관련 내용을 추론하고, 원래 문서의 주제를 통합해 유창한 텍스트를 생성하는 등 다양한 능력이 필요합니다. 또 기사를 정확하게 요약하는 방법과 법률 계약서를 요약하는 방법은 매우 다르기 때문에 정교한 수준의 도메인 일반화가 필요합니다. 이런 이유로 트랜스포머를 포함한 자연어 모델에게 텍스트 요약은 어려운 작업니다. 이런 어려움에도 불구하고 텍스트 요약은 도메인 전문가의 작업 속도를 크게 높이고 기업에서 내부 지식을 집약하고, 계약을 요약하고 소셜 미디어를 위한 자동 콘텐츠를 생성하는 등의 작업에 사용됩니다. 


이와 관련된 도전 과제를 이해하기 위해 이 장은 사전 훈련된 트랜스포머를 사용해 문서를 요약하는 방법을 알아보겠습니다. 요약은 입력과 출력이 텍스트인 고전적인 시퀀스-투-시퀀스 작업니다. 1장에서 보았듯이 요약에는 인코더-디코더 트랜스포머가 잘 맞습니다. 


이 장에서는 인코더-디코더 모델을 만들어 여러 사람이 주고 받은 대화를 간결하게 요약하겠습니다. 

하지만 그 전에 요약에 사용하는 대표적인 데이터 셋인 CNN/DAilyMail 말뭉치를 살펴보겠습니다.